# Voronoi Diagrams

## Introduction

A [Voronoi diagram](https://en.wikipedia.org/wiki/Voronoi_diagram) is a visualisation of an area partitioned into regions that minimise the distance to given point locations. These diagrams are also known variously as Voronoi tessellations, Dirichlet tessellation and Thiessen polygons. [An example](https://commons.wikimedia.org/wiki/File:Euclidean_Voronoi_diagram.svg) is shown below.

<center><img src="./media/Euclidean_Voronoi_diagram.png"width=400/>
[<a href="https://commons.wikimedia.org/wiki/File:Euclidean_Voronoi_diagram.svg" target=_blank>Source</a> | <a href="https://creativecommons.org/licenses/by-sa/4.0/deed.en" target=_blank>License</a>]</center>   

Voronoi diagrams are constructed using a similar method to buffers around points (see [Operations notebook](https://github.com/jamesdamillington/john-snow/blob/main/code/python/Operations.ipynb)), but ensure that there are no overlaps between polygons (see [Relations notebook](https://github.com/jamesdamillington/john-snow/blob/main/code/python/Relations.ipynb)). Voronoi digrams are also useful for thinking about spatial neighbourhoods (as explored in the Spatial Weights notebook). There are numerous processes [Voronoi diagrams have been used to investigate](https://en.wikipedia.org/wiki/Voronoi_diagram#Applications).  

In this notebook we will see how to create Voronoi diagrams, using ['John Snow data'](https://github.com/jamesdamillington/john-snow) about the [1854 cholera outbreak in Soho](https://en.wikipedia.org/wiki/1854_Broad_Street_cholera_outbreak). Steven Johnson discusses how Snow used Voronoi diagrams himself, in the Conclusion chapter of his [book, _The Ghost Map_](https://en.wikipedia.org/wiki/The_Ghost_Map). 

We will use functions from [the PySAL library](https://pysal.org/libpysal/)  (more examples of use [here](https://pysal.org/libpysal/notebooks/voronoi.html)) 

## Setup

First, import the necessary packages. 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from libpysal.cg.voronoi import voronoi, voronoi_frames
import numpy as np

**Note:** here were are importing `voronoi` and `voronoi_frames` functions from the `libpysal.cg` module. These are _different_ from the `Voronoi` function from `libpysal.weights` module (which we use in the Spatial Weights notebook).

Now, load data

In [ ]:
# Load point data
pumps = gpd.read_file('../../data/csds/snow7/pumps.shp')
# Load building blocks
blocks = gpd.read_file('../../data/dani/polys.shp')

Quick look at the (small) `pumps` GeoDataFrame:

In [ ]:
blocks.total_bounds

In [ ]:
pumps

And quickly plot the data to visualise what we are working with:
- blue points are locations of water pumps
- grey polygons are footprints of buildings. 

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
pumps['geometry'].plot(ax=ax)

# Create Thiessen Polygons (for a Voronoi diagram)

The `voronoi` function in requires a [_nx2 array of points_](https://pysal.org/libpysal/_modules/libpysal/cg/voronoi.html). In plain language, this means the data to create the voronoi diagram need to be provided as a 2-dimensional ( _nx2_ ) table ( _array_ ) of coordinates for point locations ( _points_ ) in the form of a [numpy array](https://numpy.org/doc/stable/reference/arrays.ndarray.html). 

We can create a numpy array from our GeoDataFrame using [`np.vstack`](https://numpy.org/doc/stable/reference/generated/numpy.vstack.html). For example:

In [ ]:
points = np.vstack([pumps['x'], pumps['y']])

Let's check the type:

In [ ]:
type(points)

And as we have only five points, we can view them all

In [ ]:
points

This is close to what we want, but rather than all the x co-ords in one column, and all the y co-ords in another, we want the co-ordinates grouped together. So get this, we need to [transpose](https://en.wikipedia.org/wiki/Transpose) the data. [This animated gif](https://commons.wikimedia.org/wiki/File:Matrix_transpose.gif) might help to visualise what a transposition does:  

<center><img src="./media/Matrix_transpose.gif" width=150/></center>

We can transpose a `np.ndarray` using the `.T` [method](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.T.html):

In [ ]:
points = np.vstack([pumps['x'], pumps['y']]).T  

Look at the data and see how they co-ords are grouped differently:

In [ ]:
points

Now we're ready to create the voronoi:

In [ ]:
results = voronoi(points)

The `voronoi` function returns a `tuple` of (`list`, `np.ndarray`):

In [ ]:
type(results)

In [ ]:
type(results[0])

In [ ]:
type(results[1])

In [ ]:
results

To get the list and array in separate objects, we can return the `voronoi` function to two objects: 

In [ ]:
regions, vertices = voronoi(points)

In [ ]:
print(type(regions))
print(regions)

In [ ]:
print(type(vertices))
print(vertices)

We could also use the `voronoi_frames` function. This does excatly the same as `vornoi`, but returns a tuple of two `GeoDataFrame`s (assuming geopandas loaded):

In [ ]:
results_df = voronoi_frames(points)

In [ ]:
type(results_df)

In [ ]:
type(results_df[0])

In [ ]:
type(results_df[1])

In [ ]:
regions_df, vertices_df = voronoi_frames(points)

In [ ]:
regions_df

In [ ]:
vertices_df

With the simple use of `voronoi` and `voronoi_frames` used above, the extent of the voronoi diagram is limited to the bounding box of the points used to create the diagram:

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
regions_df.plot(ax=ax, color='lightblue',edgecolor='black', alpha=0.3)
vertices_df.plot(ax=ax, color='red')

If we wanted to create the diagram over a larger extent, we can provide a value to the `clip` argument to the `voronoi_frames`. For example, this could be a polygon object or we could specify no clipping whatsoever with _'none'_ : 

In [ ]:
regions_df_noclip, vertices_df_noclip = voronoi_frames(points, clip='none')

f, ax = plt.subplots(1, figsize=(9, 9))
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
regions_df_noclip.plot(ax=ax, color='lightblue',edgecolor='black', alpha=0.3)
vertices_df.plot(ax=ax, color='red')

# Plotting

If we specify no clipping, we could then visualise by specifying the limits of matplotlib axes. For example:

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))

ax.set_xlim(blocks.total_bounds[0],blocks.total_bounds[2])   #use bounding box of blocks
ax.set_ylim(blocks.total_bounds[1],blocks.total_bounds[3])   #use bounding box of blocks

blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
regions_df_noclip.plot(ax=ax, color='lightblue',edgecolor='black', alpha=0.3)
vertices_df_noclip.plot(ax=ax, color='red')

We could also play with coloring the regions. 

To this, we would first add an ID column to use as a 'column' variable (like for a choropleth map):

In [ ]:
regions_df_noclip['ID'] = range(0, len(regions_df_noclip))

Then, we need to re-set the geometry:

In [ ]:
regions_df_noclip = regions_df_noclip.set_geometry('geometry')

Then we can plot using the _ID_ column to shade (using the _Set1_ colourmap):

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
ax.set_xlim(blocks.total_bounds[0],blocks.total_bounds[2])   #use bounding box of blocks
ax.set_ylim(blocks.total_bounds[1],blocks.total_bounds[3])   #use bounding box of blocks
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)

#use column here with a colourmap 
regions_df_noclip.plot(ax=ax, column='ID', cmap='Set1', edgecolor='black', alpha=0.3)
vertices_df_noclip.plot(ax=ax, color='red')

Think about how these regions are more or less useful than a simple (circular) buffer for understanding which pump people in Soho might have visited to get their water in 1854. 

# Credits!

## Contributors:
The following individual(s) have contributed to these teaching materials: James Millington (james.millington@kcl.ac.uk).

## License
These teaching materials are licensed under a mix of [The MIT License](https://opensource.org/licenses/mit-license.php) and the [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 license](https://creativecommons.org/licenses/by-nc-sa/4.0/).